In [1]:
import sys
import os
import numpy as np
import time
from imp import reload

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
    
%matplotlib qt

new directory is: C:\Users\Luke\Desktop\Barracuda\AutomatedCE


In [2]:
from L1 import Controllers
from L2 import ZControl
from L2 import CameraControl
CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
from L3 import SystemsBuilder
from testing.inputs import *
from L4 import DisplayWindows

reload(SystemsBuilder)
reload(ZControl)
reload(Controllers)
reload(DisplayWindows)
reload(CameraControl)

 Change System Config setting if thorlabs is needed. 


C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


 Change System Config setting if thorlabs is needed. 


<module 'L2.CameraControl' from 'C:\\Users\\Luke\\Desktop\\Barracuda\\AutomatedCE\\L2\\CameraControl.py'>

In [ ]:
config = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\config\NikonEclipseTi-NoLight.cfg"
ctl = Controllers.PycromanagerController(config = config)
ctl.open()

In [ ]:
try:
    cam.shutdown()
except:
    pass
cam = CameraControl.PycromanagerControl(ctl, 'image')
cam.startup()
cam.set_exposure(10)

z_ctl = ZControl.PycromanagerZ(ctl, 'objective', offset=0)

In [5]:
ce_system = SystemsBuilder.CESystem()
ce_system.camera=cam
disp = DisplayWindows.PLTMicroscopeDisplay(ce_system)
disp.show()

C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\skimage\exposure\exposure.py:351: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)


In [6]:
disp.live_image()

In [7]:
disp.stop_live_image()

We would look to see how quickly the camera can move in. So we will send a command to move the microscoe, snap a picture, then move the microscope again. 

In [161]:
distance = 0.005 
sleep = 1

get_my_image(cam, images)
move_time = time.time()
z_ctl.set_rel_z(distance)
for i in range(10):
    get_my_image(cam, images)
    time.sleep(0.1)

time.sleep(sleep)
get_my_image(cam, images)

In [8]:
from queue import Queue
import threading
import time
images = Queue()

def get_my_image(cm, img_q):
    def _update(c,i):
        snap = time.time()
        i.put([c.snap(), snap])
    threading.Thread(target=_update, args=(cm, img_q)).start()
    
def new_image(img):
    images.put([img, time.time()])
    

In [12]:
cam.update_frequency

16

In [84]:
all_data = {'time':[], 'range':[], 'distance':[]}
cam.continuous_snap()
distances = [1,3,5,10,20,40,60,100]
for x in distances:
    distance = x/1000
    for i in range(20):

        images= Queue()
        sleep = 2

        cam._callbacks = []
        cam.add_callback(new_image)
        cam.continuous_snap()
        time.sleep(0.6)
        move_time = time.time()
        z_ctl.set_rel_z(distance)

        time.sleep(sleep)
        cam._callbacks = []
        cam.stop()
        z_ctl.set_rel_z(-distance)
        time.sleep(0.6)
        time_points = []
        imgs = []
        for _ in range(images.qsize()):
            in_data = images.get()
            imgs.append( in_data[0].astype(np.float))
            time_points.append(in_data[1])
        time_points = [ x- time_points[0] for x in time_points]

        rn = []
        old_i = imgs[0]
        for i in imgs[1:]:
            df = np.subtract(i,old_i)
            rn.append(np.max(df.flatten())-np.min(df.flatten()))
        all_data['time'].append(time_points[1:])
        all_data['range'].append(rn[:])
        all_data['distance'].append([distance]*len(rn))

In [67]:
runs = len(all_data['time'])
print(runs)

160


In [ ]:
pd.concat

In [85]:
import pandas as pd
all_df = None
for rep in range(runs):
    tm = all_data['time'][rep]
    tm = np.subtract(tm,tm[np.argmax(np.gradient(tm))])
    new_data = {'time':all_data['time'][rep], 'range':all_data['range'][rep], 'distance':all_data['distance'][rep]}
    new_df = pd.DataFrame.from_dict(new_data)
    #new_df = pd.DataFrame([[all_data['time'][rep],all_data['range'][rep],all_data['distance'][rep]]], columns=['time','range','distance'])
    if all_df is None:
        all_df = new_df
    else:
        all_df = all_df.append(new_df, ignore_index=True)

In [86]:
all_df['rounded_time']=round(all_df['time']*5,0)/5

In [62]:
import seaborn as sns
sns.set(style="darkgrid")


In [88]:
sns.lineplot(x="rounded_time", y="range", hue='distance', data=all_df)

In [76]:

temp = all_df[all_df['distance']==.02]

In [77]:
temp.time

827     0.26
828     0.49
829     0.74
830     1.00
831     1.21
        ... 
1099    2.42
1100    2.64
1101    2.90
1102    3.12
1103    3.36
Name: time, Length: 277, dtype: float64

In [19]:
import matplotlib.pyplot as plt

In [193]:
cp = 5
plt.imshow(np.subtract(imgs[cp],imgs[cp-1]))
print(f"Difference is {time_points[cp]-time_points[cp-1]}")

Difference is 0.015956878662109375


In [194]:
time_points[3]-time_points[1]

0.039893388748168945

In [134]:
import numpy as np

In [212]:
rn = []
old_i = imgs[0]
for i in imgs[1:]:
    df = np.subtract(i,old_i)
    rn.append(np.max(df.flatten())-np.min(df.flatten()))
    threading.Thread(target=get_my_image, args=(cam, images)).start()

In [213]:
plt.figure()
plt.plot(time_points[1:],rn)

Exception in thread Thread-275:
Traceback (most recent call last):
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-174-cabd943a57f8>", line 9, in _update
    i.put([c.snap(), snap])
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L2\CameraControl.py", line 100, in snap
    img = self.controller.send_command(self.controller.core.get_image)
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\Controllers.py", line 226, in send_command
    raise e
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\Controllers.py", line 218, in send_command
    ans = command(*args)
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\pycromanager\core.py", line 263, in <lambda>
    fn = lambda instance, *args, signatures_list=tuple(methods_with_name): instance._translate_call(signat

Exception in thread Thread-283:
Traceback (most recent call last):
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-174-cabd943a57f8>", line 9, in _update
    i.put([c.snap(), snap])
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L2\CameraControl.py", line 100, in snap
    img = self.controller.send_command(self.controller.core.get_image)
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\Controllers.py", line 226, in send_command
    raise e
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\Controllers.py", line 218, in send_command
    ans = command(*args)
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\pycromanager\core.py", line 263, in <lambda>
    fn = lambda instance, *args, signatures_list=tuple(methods_with_name): instance._translate_call(signat

Exception in thread Thread-271:
Traceback (most recent call last):
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-174-cabd943a57f8>", line 9, in _update
    i.put([c.snap(), snap])
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L2\CameraControl.py", line 100, in snap
    img = self.controller.send_command(self.controller.core.get_image)
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\Controllers.py", line 226, in send_command
    raise e
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\Controllers.py", line 218, in send_command
    ans = command(*args)
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\pycromanager\core.py", line 263, in <lambda>
    fn = lambda instance, *args, signatures_list=tuple(methods_with_name): instance._translate_call(signat

In [205]:
disp.show()

C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\skimage\exposure\exposure.py:351: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)


In [206]:
disp.live_image()